# Manual catalog verification

This notebook presents methods for verifying that a directory contains a valid HATS catalog and performing manual verification through inspecting the catalog metadata and contents.

## Directory verification

The HATS library provides a method to verify that a directory contains the appropriate metadata files.

There are a few flavors of the validation, and the quickest one doesn't take any additional flags:

In [ ]:
from hats.io.validation import is_valid_catalog
import hats
from upath import UPath

gaia_catalog_path = UPath("https://data.lsdb.io/hats/gaia_dr3/gaia/")
is_valid_catalog(gaia_catalog_path)

### Explaining the input and output

The `strict` argument takes us through a different code path that rigorously tests the contents of all ancillary metadata files and the consistency of the partition pixels.

Here, we use the `verbose=True` argument to print out a little bit more information about our catalog. It will repeat the path that we're looking at, display the total number of partitions, and calculate the approximate sky coverage, based on the area of the HATS tiles.

The `fail_fast` argument will determine if we break out of the method at the first sign of trouble or keep looking for validation problems. This can be useful if you're debugging multiple points of failure in a catalog.

In [ ]:
is_valid_catalog(gaia_catalog_path, verbose=True, fail_fast=False, strict=True)

## Columns and data types

HATS tables are backed by parquet files. These files store metadata about their columns, the data types, and even the range of values.

The columns and types are stored on the `catalog.schema` attribute with a `pyarrow.Schema` object. You can find more details on this object and its use [in the pyarrow documents](https://arrow.apache.org/docs/python/generated/pyarrow.Schema.html)

Gaia has a lot of columns, so this display is long!

In [ ]:
catalog_object = hats.read_hats(gaia_catalog_path)
catalog_object.schema

### Column statistics

Parquet maintains basic statistics about the data inside its files. This includes the minimum value, maximum value, and the number of null (None, or unspecified) rows for that column.

We provide a method that consumes all of the min, max, and null counts, and provides global values of min and max, and a total sum of the null counts.

In [ ]:
catalog_object.aggregate_column_statistics()

Again, gaia has a lot of columns. To make the most of this output, you can either use a pandas option to display all of the rows:

```
import pandas as pd
pd.set_option('display.max_rows', None)
```

Or restrict the columns to those you care about with a keyword argument:

In [ ]:
catalog_object.aggregate_column_statistics(include_columns=["ra", "dec", "ref_epoch"])